In [256]:
#kniznice
install.packages("tidyverse")
library(tidyverse) 
library(reshape2)

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [257]:
#Pocty prenesenych dat:

#set the working directory from which the files will be read from
setwd("data/poctyPrenesenychDat/")

#create a list of the files from your target directory
file_list <- list.files(pattern="*.csv") #musel som pridat nuly pred cisla v nazvoch lebo inak bralo 1,10,11,...

#initiate a blank data frame, each iteration of the loop will append the data from the given file to this variable
dataset_poctyPrenesenychDat <- data.frame()

for (i in 1:length(file_list)){
  temp_data <- read.csv(file = file_list[i], sep=",", encoding="UTF-8") 
  dataset_poctyPrenesenychDat <- rbind(dataset_poctyPrenesenychDat, temp_data) #for each iteration, bind the new data to the building dataset
}

#Data su od 24.5.2018 do 29.2.2020. Prvy problem je, ze v podstate nemame udaje pre obdobie pocas korony,
#druhy problem je, ze v niektore dni je viac merani. Treba to spojit v ramci jednotlivych dni. Navrhujem
#predkoronove udaje neskor vygenerovat

#write.csv2(dataset, "../../dataset.csv")

setwd("../..")

#column names na lowercase
names(dataset_poctyPrenesenychDat) <- tolower(names(dataset_poctyPrenesenychDat))
#show(dataset_poctyPrenesenychDat)


In [258]:
#Udaje o pouzivateloch + dlzka pripojenia:

setwd("data/udajeDlzkaPripojenia/")

file_list <- list.files(pattern="*.csv") 

dataset_udajeDlzkaPripojenia <- data.frame()

for (i in 1:length(file_list)){
  temp_data <- read.csv(file = file_list[i], sep=";", encoding="UTF-8") 
  dataset_udajeDlzkaPripojenia <- rbind(dataset_udajeDlzkaPripojenia, temp_data) 
}
#Data su od 1.6.2018 do 25.2.2021. Problemy som nezaznamenal ziadne. Toto bude fajn zaklad pre nas dataset.

#write.csv2(dataset, "../../dataset.csv")

setwd("../..")

#column names na lowercase
names(dataset_udajeDlzkaPripojenia) <- tolower(names(dataset_udajeDlzkaPripojenia))
#show(dataset_udajeDlzkaPripojenia)



In [259]:
#Udaje o zariadeni - Jazyk, OS, Typ zariadenia, Typ prehliadaca:

setwd("data/osJazykTypPrehliadac/")

file_list <- list.files(pattern="*.csv") 

dataset_osJazykTypPrehliadac <- data.frame()

for (i in 1:length(file_list)){
  temp_data <- read.csv(file = file_list[i], sep=";", encoding="UTF-8") 
  dataset_osJazykTypPrehliadac <- rbind(dataset_osJazykTypPrehliadac, temp_data) 
}

#column names na uppercase
names(dataset_osJazykTypPrehliadac) <- tolower(names(dataset_osJazykTypPrehliadac))


#Data su od 24.5.2018 do 25.2.2021. Velmi pekne data, ktore urcite chceme pouzit. Problem je ich forma.
#Je tam variabilny pocet zaznamov pre kazdy den, podla mnozstva identifikovanych jazykov atd v dany den.
#Treba to previest na stlpce a tiez asi zvazit ktore hodnoty nam stoji za to pouzit v datasete a ktore nie. 

#write.csv2(dataset, "../../dataset.csv")
#show(dataset_osJazykTypPrehliadac)
setwd("../..")

In [260]:
#Pocty podla oblasti:

setwd("data/poctyPodlaOblasti/")

file_list <- list.files(pattern="*.csv")

dataset_poctyPodlaOblasti <- data.frame()

for (i in 1:length(file_list)){
  temp_data <- read.csv(file = file_list[i], sep=";", encoding="UTF-8", stringsAsFactors=FALSE) 
  dataset_poctyPodlaOblasti <- rbind(dataset_poctyPodlaOblasti, temp_data) 
}

#Vo vysledom dataframe su najprv udaje po celych mesiacoch a lokality su menej podelene. 
#Nasledne od 1.10.2019 su data po dnoch a "ostatne" je rozdelene na konkretne lokality. Navrhujem teda zatial
#napojit len od 1.10.2019 a pred tymto datumom to neskor podelime na dni tak aby bolo rozdelenie priblizne rovnake 

#column names na uppercase
names(dataset_poctyPodlaOblasti) <- tolower(names(dataset_poctyPodlaOblasti))


#write.csv2(dataset, "../../dataset.csv")
#show(dataset_poctyPodlaOblasti)
setwd("../..")

In [261]:
#spoji stplpce "type" a "name" => type_name
#type: browser, name: Firefox, value: 5 => type_name: browser_Firefox, value:5
type_and_name <- unite(dataset_osJazykTypPrehliadac, type_name, c(type, name), remove=TRUE)

# prekonvertuje riadok na stlpec, unique ID je date pri dataset_osJazykTypPrehliadac
# type_name: browser_Firefox, value:5 => browser_Firefox: 5 pre dany den
dataset_type_and_name <- dcast(type_and_name, date ~ type_name, value.var = "value")

# prekonvertuje riadok na stlpec, unique ID je date pri poctyPodlaOblasti
dataset_location <- dcast(dataset_poctyPodlaOblasti, date ~ location, value.var = "value")

In [262]:
#konvertujeme na Date format aby sme mohli nasledne filtrovat
dataset_location$date <- as.Date(dataset_location$date, format="%Y-%m-%d %H:%M:%OS")

#chceme data po 2019-10-01 kedze predtym data boli za cely mesiac 
dataset_filtered_location <- subset(dataset_location, date > "2019-10-01")


In [263]:
#merge datasets na zaklade date (poctyPrenesenychDat + udajeDlzkaPripojenia)
merged_dataset_1 <- merge(dataset_poctyPrenesenychDat, dataset_udajeDlzkaPripojenia, by="date", all=TRUE)

#merge datasets na zaklade date (poctyPrenesenychDat + udajeDlzkaPripojenia + osJazykTypPrehliadac)
merged_dataset_2 <- merge(merged_dataset_1, dataset_type_and_name, by="date", all=TRUE)

#merge datasets na zaklade date (poctyPrenesenychDat + udajeDlzkaPripojenia + osJazykTypPrehliadac + poctyPodlaOblasti)
final_merged_dataset <- merge(dataset_filtered_location, merged_dataset_2, by="date", all=TRUE)

#write.csv2(final_merged_dataset, "../../dataset_final.csv")
head(final_merged_dataset)



date,CTUR,FJNN (oblasť Fajnorovho Nábrežia pred SPŠS),"HLVN (oblasť Hlavného Námestia, Františkánskeho Námestia a Starej Radnice)",HURN (oblasť Hurbanovho námestia),HVZN (oblasť Hviezdoslavovho Námestia v okolí ambasády USA),KLOB (oblasť Klobučníckej a Námestia SNP pred Starou Tržnicou),KNPM (oblasť Jedlíkovej ulice),KOLN (oblasť spodnej časti Kollárovho námestia),NLST (oblasť Námestia Ľudovíta Štúra),...,lang_vi,lang_zh,os_\N,os_Android,os_iOS,os_Linux,os_Nan,os_OS X,os_others,os_Windows
2018-05-24,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,2,NA,246,193,NA,1,1,3,6
2018-05-25,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,777,479,NA,NA,NA,NA,5
2018-05-26,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,706,804,NA,NA,2,NA,NA
2018-05-27,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,2,NA,NA,828,845,NA,NA,NA,NA,6
2018-05-28,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,2,NA,490,490,NA,NA,2,NA,NA
2018-05-29,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,910,385,NA,NA,NA,NA,NA


In [264]:
#remove repeating dates

unique_dates_dataset = data.frame()

prev_row = final_merged_dataset[1,]

for(i in 2:nrow(final_merged_dataset)) 
{
    row <- final_merged_dataset[i,]
    if(row['date'] == prev_row['date'])
    {
        prev_row['data_in'] = prev_row['data_in'] + row['data_in']
        prev_row['data_out'] = prev_row['data_out'] + row['data_out']
        prev_row['unique_conn'] = prev_row['unique_conn'] + row['unique_conn']
    }
    else    
    {            
        unique_dates_dataset = rbind(unique_dates_dataset, prev_row)
            
        prev_row = row
        if(is.na(prev_row['data_in']))
            prev_row['data_in'] = 0
        if(is.na(prev_row['data_out']))
            prev_row['data_out'] = 0
        if(is.na(prev_row['unique_conn']))
            prev_row['unique_conn'] = 0        
    }
}

unique_dates_dataset = rbind(unique_dates_dataset, prev_row)

In [265]:
#drop columns containing NAs only
unique_dates_dataset['Ostatné'] <- list(NULL)
unique_dates_dataset['device_type_\\N'] <- list(NULL)
unique_dates_dataset['device_type_'] <- list(NULL)
unique_dates_dataset['lang_\\N'] <- list(NULL)
unique_dates_dataset['os_\\N'] <- list(NULL)
unique_dates_dataset['browser_\\N'] <- list(NULL)
unique_dates_dataset['Test'] <- list(NULL)

In [266]:
#replace NA with 0 where relevant

naToZeroC <- function(x) 
{   
    sapply(x, naToZeroR)
}

naToZeroR <- function(x) 
{
    if(is.na(x))
       x = 0
    x
}

df = cbind(unique_dates_dataset[1:35], lapply(unique_dates_dataset[36:ncol(unique_dates_dataset)], naToZeroC))                                                                 

In [267]:
#zlucenie jazykov

#vyber jazyky
dfa = df[58 :ncol(df)-7]

#odloz mimo jazyky ktore chceme zvlast
dfen = dfa['lang_en']
dfa['lang_en'] <- list(NULL)
dfhu = dfa['lang_hu']
dfa['lang_hu'] <- list(NULL)
dfru = dfa['lang_ru']
dfa['lang_ru'] <- list(NULL)
dfpl = dfa['lang_pl']
dfa['lang_pl'] <- list(NULL)
dfde = dfa['lang_de']
dfa['lang_de'] <- list(NULL)
dfuk = dfa['lang_fr']
dfa['lang_fr'] <- list(NULL)
dfes = dfa['lang_es']
dfa['lang_es'] <- list(NULL)
dfit = dfa['lang_it']
dfa['lang_it'] <- list(NULL)
dfsk = dfa['lang_sk']
dfa['lang_sk'] <- list(NULL)

#urob sucty ostatnych jazykov do jedneho stlpca a pripoj k stlpcom co boli pred jazykmi
dfb = cbind(df[1:57], lang_other = apply(dfa, 1, sum))

#pripoj jazyky co chceme zvlast
dfb = cbind(dfb, dfen)
dfb = cbind(dfb, dfhu)
dfb = cbind(dfb, dfru)
dfb = cbind(dfb, dfpl)
dfb = cbind(dfb, dfde)
dfb = cbind(dfb, dfuk)
dfb = cbind(dfb, dfes)
dfb = cbind(dfb, dfit)
dfb = cbind(dfb, dfsk)

#pripoj stlpce co boli za jazykmi
dfb = cbind(dfb, df[(ncol(df)-6):(ncol(df))])


In [268]:
write.csv2(dfb, "../../dfb.csv")